In [1]:
import pandas as pd
import numpy as np

In [2]:
# change to your file path
df = pd.read_csv('Microsoft_Stock.csv')
df.head()

,Date,Open,High,Low,Close,Volume
0,4/1/2015 16:00:00,40.60,40.76,40.31,40.72,36865322
1,4/2/2015 16:00:00,40.66,40.74,40.12,40.29,37487476
2,4/6/2015 16:00:00,40.34,41.78,40.18,41.55,39223692
3,4/7/2015 16:00:00,41.61,41.91,41.31,41.53,28809375
4,4/8/2015 16:00:00,41.48,41.69,41.04,41.42,24753438


In [3]:
from datetime import datetime

# convert the string to datetime object
df['Date'] = [datetime.strptime(date, "%m/%d/%Y %H:%M:%S") for date in df['Date']]

# Convert the datetime into InfluxDB format
df['Date'] = [date.strftime("%Y-%m-%dT%H:%M:%SZ") for date in  df['Date']]

df.head()


,Date,Open,High,Low,Close,Volume
0,2015-04-01T16:00:00Z,40.60,40.76,40.31,40.72,36865322
1,2015-04-02T16:00:00Z,40.66,40.74,40.12,40.29,37487476
2,2015-04-06T16:00:00Z,40.34,41.78,40.18,41.55,39223692
3,2015-04-07T16:00:00Z,41.61,41.91,41.31,41.53,28809375
4,2015-04-08T16:00:00Z,41.48,41.69,41.04,41.42,24753438


In [4]:
# stock price is what we are measuring
df['measurement'] = ['Stock_Price'] * df.shape[0]

In [5]:
# the tag here is like a company
df['tag'] = ['Microsoft'] * df.shape[0]

In [6]:
# this seems weird but it is neccessary for annotation format
# empty row to filled about the "" column
df[''] = [''] * df.shape[0]

In [7]:
# empty row to filled about the "result" column
df['result'] = [''] * df.shape[0]

In [8]:
# table is also required for annotation format
df['table'] = [0] * df.shape[0]

In [9]:
# gather the data together
df = df[['', 'result', 'table', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'measurement', 'tag']]

In [10]:
# it looks good now but still far away from what we expect
df.head()

,,result,table,Date,Open,High,Low,Close,Volume,measurement,tag
0,,,0,2015-04-01T16:00:00Z,40.60,40.76,40.31,40.72,36865322,Stock_Price,Microsoft
1,,,0,2015-04-02T16:00:00Z,40.66,40.74,40.12,40.29,37487476,Stock_Price,Microsoft
2,,,0,2015-04-06T16:00:00Z,40.34,41.78,40.18,41.55,39223692,Stock_Price,Microsoft
3,,,0,2015-04-07T16:00:00Z,41.61,41.91,41.31,41.53,28809375,Stock_Price,Microsoft
4,,,0,2015-04-08T16:00:00Z,41.48,41.69,41.04,41.42,24753438,Stock_Price,Microsoft


In [11]:
# here we extract the feats columns that represents Stock price (in order words, open, high, low...) into rows
# so that we can combine them into a single column

open = df['Open'].to_numpy().reshape((-1, 1))
high = df['High'].to_numpy().reshape((-1, 1))
low = df['Low'].to_numpy().reshape((-1, 1))
close = df['Close'].to_numpy().reshape((-1, 1))
volume = df['Volume'].to_numpy().reshape((-1, 1))

value = np.vstack([open, high, low, close, volume])

# how to distinguish which number is open stock value and which is close?
# we creates labels for them, and again, combine into a single column
n = df.shape[0]

open = np.array(['open' for i in range(n)]).reshape((-1, 1))
high = np.array(['high' for i in range(n)]).reshape((-1, 1))
low = np.array(['low' for i in range(n)]).reshape((-1, 1))
close = np.array(['close' for i in range(n)]).reshape((-1, 1))
volume = np.array(['volume' for i in range(n)]).reshape((-1, 1))

field = np.vstack([open, high, low, close, volume])

In [13]:
# now we make an empty dataframe to be filled in
updated_df = pd.DataFrame(columns=['', 'result', 'table', '_time', '_value', '_field', '_measurement', 'company'])

In [14]:
# unchanged df contains all the information of the original data except the feats we just extracts
unchanged = df[['', 'result', 'table', 'Date', 'measurement', 'tag']].to_numpy()
# we stacked five times because there is five feats
updated_df[['', 'result', 'table', '_time', '_measurement', 'company']] = np.vstack([unchanged, unchanged, unchanged, unchanged, unchanged])

In [15]:
updated_df.head()

,,result,table,_time,_value,_field,_measurement,company
0,,,0,2015-04-01T16:00:00Z,NaN,NaN,Stock_Price,Microsoft
1,,,0,2015-04-02T16:00:00Z,NaN,NaN,Stock_Price,Microsoft
2,,,0,2015-04-06T16:00:00Z,NaN,NaN,Stock_Price,Microsoft
3,,,0,2015-04-07T16:00:00Z,NaN,NaN,Stock_Price,Microsoft
4,,,0,2015-04-08T16:00:00Z,NaN,NaN,Stock_Price,Microsoft


Don't forget the value and field we created before

In [16]:
updated_df['_value'] = value
updated_df['_field'] = field

In [17]:
# make sure the data still have a time order
updated_df = updated_df.sort_values(by=['_time'])

In [18]:
updated_df.head()

,,result,table,_time,_value,_field,_measurement,company
0,,,0,2015-04-01T16:00:00Z,40.60,open,Stock_Price,Microsoft
1511,,,0,2015-04-01T16:00:00Z,40.76,high,Stock_Price,Microsoft
3022,,,0,2015-04-01T16:00:00Z,40.31,low,Stock_Price,Microsoft
4533,,,0,2015-04-01T16:00:00Z,40.72,close,Stock_Price,Microsoft
6044,,,0,2015-04-01T16:00:00Z,36865322.00,volume,Stock_Price,Microsoft


In [19]:
updated_df.to_csv('prac_Microsoft_stock.csv',index=False)

We are not done yet! We just turn our data to desired format, but we also need to annotate the data. Copy and paste the annotation below to the start of your csv file! Make sure you get rid of the """ """.

In [20]:
annoataion = """#group,false,false,false,false,true,true,true
#datatype,string,long,dateTime:RFC3339,double,string,string,string
#default,_result,,,,,,
,result,table,_time,_value,_field,_measurement,company"""